In [84]:
import pandas as pd
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb
from PIL import Image
import glob

### training data preparation

In [85]:
img_all = glob.glob(r'train/*.jpg')
img_all

['train\\apple_1.jpg',
 'train\\apple_10.jpg',
 'train\\apple_11.jpg',
 'train\\apple_12.jpg',
 'train\\apple_13.jpg',
 'train\\apple_14.jpg',
 'train\\apple_15.jpg',
 'train\\apple_16.jpg',
 'train\\apple_17.jpg',
 'train\\apple_18.jpg',
 'train\\apple_19.jpg',
 'train\\apple_2.jpg',
 'train\\apple_20.jpg',
 'train\\apple_21.jpg',
 'train\\apple_22.jpg',
 'train\\apple_23.jpg',
 'train\\apple_24.jpg',
 'train\\apple_25.jpg',
 'train\\apple_26.jpg',
 'train\\apple_27.jpg',
 'train\\apple_28.jpg',
 'train\\apple_29.jpg',
 'train\\apple_3.jpg',
 'train\\apple_30.jpg',
 'train\\apple_31.jpg',
 'train\\apple_32.jpg',
 'train\\apple_33.jpg',
 'train\\apple_34.jpg',
 'train\\apple_35.jpg',
 'train\\apple_36.jpg',
 'train\\apple_37.jpg',
 'train\\apple_38.jpg',
 'train\\apple_39.jpg',
 'train\\apple_4.jpg',
 'train\\apple_40.jpg',
 'train\\apple_41.jpg',
 'train\\apple_42.jpg',
 'train\\apple_43.jpg',
 'train\\apple_44.jpg',
 'train\\apple_45.jpg',
 'train\\apple_46.jpg',
 'train\\apple_47.jp

In [86]:
np.array(img_all).shape

(248,)

In [87]:
# prepare the feature
x_train = None
for i in img_all:
    img = Image.open(i).convert('RGB')
    img_resize = img.resize((64,64))
    if x_train is None:
        x_train = img_resize
    else:
        x_train = np.concatenate((x_train, img_resize))
print(x_train.shape)
x_train = x_train.reshape(-1,64,64,3)
x_train.shape

(15872, 64, 3)


(248, 64, 64, 3)

In [88]:
# label: 1hot-encode: [1,0,0]-apple  [0,1,0]-banana  [0,0,1]-orange  [1,1,1]-mixed
y = []
for i in img_all:
    if 'apple' in i:
        y.append([1,0,0])
    if 'banana' in i:
        y.append([0,1,0])
    if 'orange' in i:
        y.append([0,0,1])
    if 'mixed' in i:
        y.append([1,1,1])
y_train = np.array(y)
y_train.shape

(248, 3)

### create and train model

In [89]:
model_1 = tf.keras.Sequential()

model_1.add(tf.keras.layers.Conv2D(filters=32,
    kernel_size=(3, 3),
    activation='relu',
    input_shape=(64, 64, 3)))
model_1.add(tf.keras.layers.Conv2D(filters=32,
    kernel_size=(5, 5),
    activation='relu',
    input_shape=(64, 64, 3)))
model_1.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
model_1.add(tf.keras.layers.Dropout(rate=0.25))
model_1.add(tf.keras.layers.Flatten())
model_1.add(tf.keras.layers.Dense(units=64, activation='relu'))
model_1.add(tf.keras.layers.Dropout(rate=0.5))
model_1.add(tf.keras.layers.Dense(units=3, activation='softmax'))

In [90]:
model_1.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [91]:
hist = model_1.fit(x=x_train/255, y=y_train, epochs=15)

Epoch 1/15
8/8 [==============================] - 1s 163ms/step - loss: 1.6866 - accuracy: 0.3710
Epoch 2/15
8/8 [==============================] - 1s 161ms/step - loss: 1.3210 - accuracy: 0.3911
Epoch 3/15
8/8 [==============================] - 1s 163ms/step - loss: 2.7259 - accuracy: 0.3427
Epoch 4/15
8/8 [==============================] - 1s 177ms/step - loss: 3.4667 - accuracy: 0.3710
Epoch 5/15
8/8 [==============================] - 1s 178ms/step - loss: 5.0687 - accuracy: 0.3871
Epoch 6/15
8/8 [==============================] - 1s 164ms/step - loss: 12.4879 - accuracy: 0.3750
Epoch 7/15
8/8 [==============================] - 1s 167ms/step - loss: 29.7899 - accuracy: 0.3468
Epoch 8/15
8/8 [==============================] - 1s 175ms/step - loss: 76.2492 - accuracy: 0.3387
Epoch 9/15
8/8 [==============================] - 1s 178ms/step - loss: 124.5259 - accuracy: 0.3710
Epoch 10/15
8/8 [==============================] - 1s 177ms/step - loss: 206.2210 - accuracy: 0.3226
Epoch 11/15


### change the 1hot-encoding pattern, and train again

In [92]:
# label: 1hot-encode: [1,0,0,0]-apple  [0,1,0,0]-banana  [0,0,1,0]-orange  [0,0,0,1]-mixed
y = []
for i in img_all:
    if 'apple' in i:
        y.append([1,0,0,0])
    if 'banana' in i:
        y.append([0,1,0,0])
    if 'orange' in i:
        y.append([0,0,1,0])
    if 'mixed' in i:
        y.append([0,0,0,1])
y_train = np.array(y)
y_train.shape

(248, 4)

In [93]:
model_2 = tf.keras.Sequential()

model_2.add(tf.keras.layers.Conv2D(filters=32,
    kernel_size=(3, 3),
    activation='relu',
    input_shape=(64, 64, 3)))
model_2.add(tf.keras.layers.Conv2D(filters=32,
    kernel_size=(5, 5),
    activation='relu',
    input_shape=(64, 64, 3)))
model_2.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
model_2.add(tf.keras.layers.Dropout(rate=0.25))
model_2.add(tf.keras.layers.Flatten())
model_2.add(tf.keras.layers.Dense(units=64, activation='relu'))
model_2.add(tf.keras.layers.Dropout(rate=0.5))
model_2.add(tf.keras.layers.Dense(units=4, activation='softmax'))
model_2.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [94]:
#x_train does not change
hist = model_2.fit(x=x_train/255, y=y_train, epochs=15)

Epoch 1/15
8/8 [==============================] - 1s 164ms/step - loss: 1.4714 - accuracy: 0.4073
Epoch 2/15
8/8 [==============================] - 1s 175ms/step - loss: 1.1035 - accuracy: 0.5645
Epoch 3/15
8/8 [==============================] - 1s 160ms/step - loss: 0.9055 - accuracy: 0.6290
Epoch 4/15
8/8 [==============================] - 1s 164ms/step - loss: 0.8205 - accuracy: 0.6935
Epoch 5/15
8/8 [==============================] - 1s 160ms/step - loss: 0.5942 - accuracy: 0.7621
Epoch 6/15
8/8 [==============================] - 1s 177ms/step - loss: 0.5516 - accuracy: 0.8024
Epoch 7/15
8/8 [==============================] - 1s 159ms/step - loss: 0.4314 - accuracy: 0.8306
Epoch 8/15
8/8 [==============================] - 1s 160ms/step - loss: 0.4065 - accuracy: 0.8427
Epoch 9/15
8/8 [==============================] - 1s 163ms/step - loss: 0.3443 - accuracy: 0.8952
Epoch 10/15
8/8 [==============================] - 1s 160ms/step - loss: 0.2862 - accuracy: 0.9073
Epoch 11/15
8/8 [==

In [95]:
# training accuracy improved a lot after adjusting the label encoding. 
# now resize the images to larger(128x128) to see the effect:
x_train = None
for i in img_all:
    img = Image.open(i).convert('RGB')
    img_resize = img.resize((128,128))
    if x_train is None:
        x_train = img_resize
    else:
        x_train = np.concatenate((x_train, img_resize))
print(x_train.shape)
x_train = x_train.reshape(-1,128,128,3)
x_train.shape

(31744, 128, 3)


(248, 128, 128, 3)

In [96]:
model_3 = tf.keras.Sequential()

model_3.add(tf.keras.layers.Conv2D(filters=32,
    kernel_size=(3, 3),
    activation='relu',
    input_shape=(128, 128, 3)))
model_3.add(tf.keras.layers.Conv2D(filters=32,
    kernel_size=(5, 5),
    activation='relu',
    input_shape=(128, 128, 3)))
model_3.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
model_3.add(tf.keras.layers.Dropout(rate=0.25))
model_3.add(tf.keras.layers.Flatten())
model_3.add(tf.keras.layers.Dense(units=128, activation='relu'))
model_3.add(tf.keras.layers.Dropout(rate=0.5))
model_3.add(tf.keras.layers.Dense(units=4, activation='softmax'))
model_3.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [97]:
# y_train does not change
hist = model_3.fit(x=x_train/255, y=y_train, epochs=20)

Epoch 1/20
8/8 [==============================] - 6s 786ms/step - loss: 2.2619 - accuracy: 0.3548
Epoch 2/20
8/8 [==============================] - 6s 773ms/step - loss: 0.9597 - accuracy: 0.6331
Epoch 3/20
8/8 [==============================] - 7s 818ms/step - loss: 0.6265 - accuracy: 0.7702
Epoch 4/20
8/8 [==============================] - 6s 768ms/step - loss: 0.4816 - accuracy: 0.8185
Epoch 5/20
8/8 [==============================] - 6s 762ms/step - loss: 0.2686 - accuracy: 0.9315
Epoch 6/20
8/8 [==============================] - 7s 822ms/step - loss: 0.2480 - accuracy: 0.9073
Epoch 7/20
8/8 [==============================] - 6s 745ms/step - loss: 0.1775 - accuracy: 0.9476
Epoch 8/20
8/8 [==============================] - 6s 750ms/step - loss: 0.0861 - accuracy: 0.9839
Epoch 9/20
8/8 [==============================] - 6s 760ms/step - loss: 0.1060 - accuracy: 0.9758
Epoch 10/20
8/8 [==============================] - 6s 764ms/step - loss: 0.0824 - accuracy: 0.9758
Epoch 11/20
8/8 [==

In [79]:
#slight improvement. ready for testing

### prepare test data

In [80]:
img_test = glob.glob(r'test/*.jpg')
img_test

['test\\apple_77.jpg',
 'test\\apple_78.jpg',
 'test\\apple_79.jpg',
 'test\\apple_80.jpg',
 'test\\apple_81.jpg',
 'test\\apple_82.jpg',
 'test\\apple_83.jpg',
 'test\\apple_84.jpg',
 'test\\apple_85.jpg',
 'test\\apple_86.jpg',
 'test\\apple_87.jpg',
 'test\\apple_88.jpg',
 'test\\apple_89.jpg',
 'test\\apple_90.jpg',
 'test\\apple_91.jpg',
 'test\\apple_92.jpg',
 'test\\apple_93.jpg',
 'test\\apple_94.jpg',
 'test\\apple_95.jpg',
 'test\\banana_77.jpg',
 'test\\banana_78.jpg',
 'test\\banana_79.jpg',
 'test\\banana_80.jpg',
 'test\\banana_81.jpg',
 'test\\banana_82.jpg',
 'test\\banana_83.jpg',
 'test\\banana_84.jpg',
 'test\\banana_85.jpg',
 'test\\banana_86.jpg',
 'test\\banana_87.jpg',
 'test\\banana_88.jpg',
 'test\\banana_89.jpg',
 'test\\banana_90.jpg',
 'test\\banana_91.jpg',
 'test\\banana_92.jpg',
 'test\\banana_93.jpg',
 'test\\banana_94.jpg',
 'test\\mixed_21.jpg',
 'test\\mixed_22.jpg',
 'test\\mixed_23.jpg',
 'test\\mixed_24.jpg',
 'test\\mixed_25.jpg',
 'test\\orange_7

In [83]:
y_truth = []
for i in img_test:
    if 'apple' in i:
        y_truth.append([1,0,0,0])
    if 'banana' in i:
        y_truth.append([0,1,0,0])
    if 'orange' in i:
        y_truth.append([0,0,1,0])
    if 'mixed' in i:
        y_truth.append([0,0,0,1])
y_truth = np.array(y_truth)
y_truth.shape

(60, 4)

In [100]:
x_test_64 = None
x_test_128 = None
for i in img_test:
    im = Image.open(i).convert('RGB')
    im_64 = im.resize((64,64))
    im_128 = im.resize((128,128))
    if x_test_64 is None:
        x_test_64 = im_64
        x_test_128 = im_128
    else:
        x_test_64 = np.concatenate((x_test_64, im_64))
        x_test_128 = np.concatenate((x_test_128, im_128))
x_test_64 = x_test_64.reshape((-1,64,64,3))
x_test_128 = x_test_128.reshape((-1,128,128,3))
print(x_test_64.shape, '\n', x_test_128.shape)

(60, 64, 64, 3) 
 (60, 128, 128, 3)


### test the models and evaluate

In [103]:
loss_64, accuracy_64 = model_2.evaluate(x=x_test_64, y=y_truth)
print('loss_64 =', loss_64)
print('accuracy_64 =', accuracy_64)

2/2 [==============================] - 0s 12ms/step - loss: 154.8541 - accuracy: 0.8500
loss_64 = 154.85411071777344
accuracy_64 = 0.8500000238418579


In [104]:
loss_128, accuracy_128 = model_3.evaluate(x=x_test_128, y=y_truth)
print('loss_128 =', loss_128)
print('accuracy_128 =', accuracy_128)

2/2 [==============================] - 0s 51ms/step - loss: 193.3968 - accuracy: 0.8833
loss_128 = 193.39683532714844
accuracy_128 = 0.8833333253860474


In [125]:
y_pred = model_3.predict(x_test_128)
correct = 0
wrong = 0
for i in range(len(y_pred)):
    print('predict:', y_pred[i], '   truth:', y_truth[i])
    if (y_pred[i].tolist() == y_truth[i].tolist()):
        correct+=1
    else:
        wrong+=1
print('correct = ', correct, '   ', 'wrong = ', wrong)
print('accuracy = ',correct/(correct+wrong))

predict: [1. 0. 0. 0.]    truth: [1 0 0 0]
predict: [1. 0. 0. 0.]    truth: [1 0 0 0]
predict: [1. 0. 0. 0.]    truth: [1 0 0 0]
predict: [1. 0. 0. 0.]    truth: [1 0 0 0]
predict: [1. 0. 0. 0.]    truth: [1 0 0 0]
predict: [1. 0. 0. 0.]    truth: [1 0 0 0]
predict: [1. 0. 0. 0.]    truth: [1 0 0 0]
predict: [1. 0. 0. 0.]    truth: [1 0 0 0]
predict: [1. 0. 0. 0.]    truth: [1 0 0 0]
predict: [1. 0. 0. 0.]    truth: [1 0 0 0]
predict: [1. 0. 0. 0.]    truth: [1 0 0 0]
predict: [1. 0. 0. 0.]    truth: [1 0 0 0]
predict: [1. 0. 0. 0.]    truth: [1 0 0 0]
predict: [1. 0. 0. 0.]    truth: [1 0 0 0]
predict: [1. 0. 0. 0.]    truth: [1 0 0 0]
predict: [1. 0. 0. 0.]    truth: [1 0 0 0]
predict: [1. 0. 0. 0.]    truth: [1 0 0 0]
predict: [1. 0. 0. 0.]    truth: [1 0 0 0]
predict: [0. 0. 1. 0.]    truth: [1 0 0 0]
predict: [0. 1. 0. 0.]    truth: [0 1 0 0]
predict: [0. 1. 0. 0.]    truth: [0 1 0 0]
predict: [0. 1. 0. 0.]    truth: [0 1 0 0]
predict: [0. 1. 0. 0.]    truth: [0 1 0 0]
predict: [0